# Carga de librerías

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import gc

In [2]:
from olympics.google_service.client import GoogleClient
from olympics.modelling.dataset import WFMDataset
import olympics.modelling.outlier_detection as OutlierDetection
import olympics.modelling.imputation as Imputation
import olympics.modelling.modeling as Modelling
import olympics.modelling.feature_engineering as Engineering

/Users/MI31539/anaconda3/envs/data_olympics/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Leer base de datos

In [3]:
df = pd.read_excel(
    io='/Users/MI31539/Desktop/data_olympics/csv/Data - Data Olympics Concurso Finanzas.xlsx',
    sheet_name='DATOS CONCURSO'
)

In [4]:
df.head()

,fecha,cetes_12m,cetes_1m,cetes_3m,cetes_6m,exchange_rate_usd,exportaciones_no_petroleras,imss,inpc,official_interest_rate_usa,...,treasury_10y,treasury_1m,treasury_1y,treasury_3m,treasury_3y,treasury_5y,treasury_6m,vix,vix_bmv,corporativa_mn
0,2007-01-01,5.998714,5.270554,7.619891,9.538785,16.626715,65523.200303,1.492044e+07,57.490374,1.385472,...,4.601301,4.511093,1.371252,3.797369,1.081369,6.241602,1.704889,7.659628,21.931550,36435.716117
1,2007-02-01,5.998714,5.270554,7.619891,9.538785,16.626715,65523.200303,1.492044e+07,57.490374,1.385472,...,4.601301,4.622545,1.371252,3.797369,1.081369,6.241602,1.704889,7.659628,20.456388,36435.716117
2,2007-03-01,7.483663,5.270554,7.619891,9.538785,16.626715,65523.200303,1.492044e+07,57.490374,1.385472,...,4.601301,4.694684,1.371252,4.805535,1.081369,6.241602,1.704889,12.211318,22.318382,36435.716117
3,2007-04-01,8.106428,5.270554,7.619891,9.538785,16.626715,65523.200303,1.492044e+07,57.490374,1.385472,...,4.601301,5.155397,1.371252,4.917340,1.081369,6.241602,1.704889,14.623812,22.722563,36435.716117
4,2007-05-01,8.232189,5.270554,7.619891,9.538785,16.626715,65523.200303,1.492044e+07,57.490374,1.385472,...,4.601301,5.076232,1.371252,4.885224,1.081369,6.241602,1.704889,14.856131,22.418558,36435.716117


In [5]:
df.columns

Index(['fecha', 'cetes_12m', 'cetes_1m', 'cetes_3m', 'cetes_6m',
       'exchange_rate_usd', 'exportaciones_no_petroleras', 'imss', 'inpc',
       'official_interest_rate_usa', 'pib', 's&p500', 'sovereign_10y',
       'sovereign_3y', 'sovereign_5y', 'stock_market', 'tasa_desempleo',
       'tasa_fondeo_1d', 'treasury_10y', 'treasury_1m', 'treasury_1y',
       'treasury_3m', 'treasury_3y', 'treasury_5y', 'treasury_6m', 'vix',
       'vix_bmv', 'corporativa_mn'],
      dtype='object')

In [6]:
df.shape

(208, 28)

In [7]:
df.dtypes

fecha                          datetime64[ns]
cetes_12m                             float64
cetes_1m                              float64
cetes_3m                              float64
cetes_6m                              float64
exchange_rate_usd                     float64
exportaciones_no_petroleras           float64
imss                                  float64
inpc                                  float64
official_interest_rate_usa            float64
pib                                   float64
s&p500                                float64
sovereign_10y                         float64
sovereign_3y                          float64
sovereign_5y                          float64
stock_market                          float64
tasa_desempleo                        float64
tasa_fondeo_1d                        float64
treasury_10y                          float64
treasury_1m                           float64
treasury_1y                           float64
treasury_3m                       

# Feature Engineering

In [8]:
def add_lags(df, col, lags):
    for lag in lags:
        df[f"{col}_lag{lag}"] = df[col].shift(lag)
    return df

def add_rolling_features(df, col, windows=[3, 6]):
    for win in windows:
        df[f"{col}rollmean{win}"] = df[col].rolling(win).mean()
        df[f"{col}diff{win}"] = df[col].diff(win)
    return df

In [10]:
def engineering(df):
    original = [x for x in df.columns[:-1]]
    target = [df.columns[-1]]
    df = add_rolling_features(df, 'corporativa_mn', [3, 6])
    df = add_lags(df, 'corporativa_mn', [3])
    df = add_lags(df, 'treasury_5y', [1])
    df = add_lags(df, 'cetes_1m', [1, 2])
    df = add_lags(df, 'imss', [2])
    new = [
        'corporativa_mnrollmean3', 'corporativa_mndiff3',
        'corporativa_mnrollmean6', 'corporativa_mndiff6',
        'corporativa_mn_lag3', 'treasury_5y_lag1', 'cetes_1m_lag1',
        'cetes_1m_lag2'
    ]
    order = original + new + target
    return df[order]

In [11]:
df = engineering(df)

In [12]:
df.isna().sum()

fecha                           0
cetes_12m                       0
cetes_1m                        0
cetes_3m                        0
cetes_6m                        0
exchange_rate_usd               0
exportaciones_no_petroleras     0
imss                            0
inpc                            0
official_interest_rate_usa      0
pib                             0
s&p500                          0
sovereign_10y                   0
sovereign_3y                    0
sovereign_5y                    0
stock_market                    0
tasa_desempleo                  0
tasa_fondeo_1d                  0
treasury_10y                    0
treasury_1m                     0
treasury_1y                     0
treasury_3m                     0
treasury_3y                     0
treasury_5y                     0
treasury_6m                     0
vix                             0
vix_bmv                         0
corporativa_mnrollmean3        26
corporativa_mndiff3            27
corporativa_mn

# Data train_test_split

In [13]:
train_ix = 172
test_ix = 184

In [14]:
X_train = df.iloc[:train_ix,:-1]
y_train = df.iloc[:train_ix,-1]
X_valid = df.iloc[train_ix:test_ix,:-1]
y_valid = df.iloc[train_ix:test_ix,-1]
X_test = df.iloc[test_ix:,:-1]
y_test = df.iloc[test_ix:,-1]

# Outliers

In [15]:
detection_factories = {
    col: OutlierDetection.MADFactory()
    for col in X_train.columns[1:]
}
detection_client = OutlierDetection.OutlierDetectionClient(column_factory_map=detection_factories, remainder='passthrough')
detection_client.fit(X_train)

In [16]:
train_outliers = detection_client.detect(X_train)
train_outliers.sum()

cetes_12m                       0
cetes_1m                        0
cetes_3m                        0
cetes_6m                        0
exchange_rate_usd               0
exportaciones_no_petroleras     0
imss                            0
inpc                            0
official_interest_rate_usa      0
pib                             0
s&p500                          0
sovereign_10y                   0
sovereign_3y                    0
sovereign_5y                    0
stock_market                    0
tasa_desempleo                  0
tasa_fondeo_1d                  0
treasury_10y                    0
treasury_1m                     3
treasury_1y                     0
treasury_3m                     5
treasury_3y                     3
treasury_5y                    16
treasury_6m                     0
vix                             0
vix_bmv                         0
corporativa_mnrollmean3         0
corporativa_mndiff3             1
corporativa_mnrollmean6         0
corporativa_mn

In [17]:
valid_outliers = detection_client.detect(X_valid)
valid_outliers.sum()

cetes_12m                      0
cetes_1m                       0
cetes_3m                       0
cetes_6m                       0
exchange_rate_usd              0
exportaciones_no_petroleras    0
imss                           0
inpc                           0
official_interest_rate_usa     0
pib                            0
s&p500                         0
sovereign_10y                  0
sovereign_3y                   0
sovereign_5y                   0
stock_market                   0
tasa_desempleo                 0
tasa_fondeo_1d                 0
treasury_10y                   0
treasury_1m                    0
treasury_1y                    0
treasury_3m                    0
treasury_3y                    0
treasury_5y                    3
treasury_6m                    0
vix                            0
vix_bmv                        0
corporativa_mnrollmean3        0
corporativa_mndiff3            0
corporativa_mnrollmean6        0
corporativa_mndiff6            0
corporativ

In [18]:
test_outliers = detection_client.detect(X_test)
test_outliers.sum()

cetes_12m                       0
cetes_1m                       15
cetes_3m                        0
cetes_6m                        0
exchange_rate_usd               0
exportaciones_no_petroleras     0
imss                            0
inpc                            0
official_interest_rate_usa      0
pib                             0
s&p500                          0
sovereign_10y                   0
sovereign_3y                    8
sovereign_5y                    0
stock_market                    0
tasa_desempleo                  0
tasa_fondeo_1d                  0
treasury_10y                    0
treasury_1m                     3
treasury_1y                    15
treasury_3m                     1
treasury_3y                    16
treasury_5y                    14
treasury_6m                    12
vix                             0
vix_bmv                         0
corporativa_mnrollmean3         0
corporativa_mndiff3             0
corporativa_mnrollmean6         0
corporativa_mn

In [19]:
X_train = detection_client.transform(X_train)
X_valid = detection_client.transform(X_valid)
X_test = detection_client.transform(X_test)

# Datos faltantes

In [20]:
X_train.isna().sum()

fecha                           0
cetes_12m                       0
cetes_1m                        0
cetes_3m                        0
cetes_6m                        0
exchange_rate_usd               0
exportaciones_no_petroleras     0
imss                            0
inpc                            0
official_interest_rate_usa      0
pib                             0
s&p500                          0
sovereign_10y                   0
sovereign_3y                    0
sovereign_5y                    0
stock_market                    0
tasa_desempleo                  0
tasa_fondeo_1d                  0
treasury_10y                    0
treasury_1m                     3
treasury_1y                     0
treasury_3m                     5
treasury_3y                     3
treasury_5y                    16
treasury_6m                     0
vix                             0
vix_bmv                         0
corporativa_mnrollmean3         2
corporativa_mndiff3             4
corporativa_mn

In [21]:
imputation_cliente = Imputation.ImputationClient()
imputation_cliente.fit(X_train)

In [22]:
X_train = imputation_cliente.transform(X_train)
X_valid = imputation_cliente.transform(X_valid)
X_test = imputation_cliente.transform(X_test)

In [23]:
X_train.isna().sum()

fecha                          0
cetes_12m                      0
cetes_1m                       0
cetes_3m                       0
cetes_6m                       0
exchange_rate_usd              0
exportaciones_no_petroleras    0
imss                           0
inpc                           0
official_interest_rate_usa     0
pib                            0
s&p500                         0
sovereign_10y                  0
sovereign_3y                   0
sovereign_5y                   0
stock_market                   0
tasa_desempleo                 0
tasa_fondeo_1d                 0
treasury_10y                   0
treasury_1m                    0
treasury_1y                    0
treasury_3m                    0
treasury_3y                    0
treasury_5y                    0
treasury_6m                    0
vix                            0
vix_bmv                        0
corporativa_mnrollmean3        0
corporativa_mndiff3            0
corporativa_mnrollmean6        0
corporativ

In [24]:
X_valid.isna().sum()

fecha                          0
cetes_12m                      0
cetes_1m                       0
cetes_3m                       0
cetes_6m                       0
exchange_rate_usd              0
exportaciones_no_petroleras    0
imss                           0
inpc                           0
official_interest_rate_usa     0
pib                            0
s&p500                         0
sovereign_10y                  0
sovereign_3y                   0
sovereign_5y                   0
stock_market                   0
tasa_desempleo                 0
tasa_fondeo_1d                 0
treasury_10y                   0
treasury_1m                    0
treasury_1y                    0
treasury_3m                    0
treasury_3y                    0
treasury_5y                    0
treasury_6m                    0
vix                            0
vix_bmv                        0
corporativa_mnrollmean3        0
corporativa_mndiff3            0
corporativa_mnrollmean6        0
corporativ

In [25]:
X_test.isna().sum()

fecha                          0
cetes_12m                      0
cetes_1m                       0
cetes_3m                       0
cetes_6m                       0
exchange_rate_usd              0
exportaciones_no_petroleras    0
imss                           0
inpc                           0
official_interest_rate_usa     0
pib                            0
s&p500                         0
sovereign_10y                  0
sovereign_3y                   0
sovereign_5y                   0
stock_market                   0
tasa_desempleo                 0
tasa_fondeo_1d                 0
treasury_10y                   0
treasury_1m                    0
treasury_1y                    0
treasury_3m                    0
treasury_3y                    0
treasury_5y                    0
treasury_6m                    0
vix                            0
vix_bmv                        0
corporativa_mnrollmean3        0
corporativa_mndiff3            0
corporativa_mnrollmean6        0
corporativ

# Modelado

In [53]:
feature_sets = [
    [
        "cetes_1m", "treasury_6m", "cetes_3m", "sovereign_3y",
        "cetes_6m", "cetes_12m", "tasa_fondeo_1d", "sovereign_5y",
        "treasury_1m", "treasury_3m", "sovereign_10y", "treasury_3y",
        "official_interest_rate_usa", "treasury_1y"
    ],
    [
        "imss", "s&p500", "vix_bmv", "inpc", "exchange_rate_usd",
        "tasa_desempleo"
    ],
    [
        "treasury_10y", "treasury_5y", "vix", "stock_market",
        "exportaciones_no_petroleras"
    ],
    [
        'corporativa_mnrollmean3', 'corporativa_mndiff3',
        'corporativa_mnrollmean6', 'corporativa_mndiff6',
        'corporativa_mn_lag3', 'treasury_5y_lag1', 'cetes_1m_lag1',
        'cetes_1m_lag2'
    ]
]
feature_sets = feature_sets[2:]

In [89]:
base_clients = [
    Modelling.ModelClient(
        'linearregression',
        #n_estimators=150,
        #max_depth=10,
        #learning_rate=0.1,
        #random_state=42
    )
    for _ in feature_sets
]
bases = list(zip(base_clients, feature_sets))

In [90]:
meta_client = Modelling.ModelClient('linearregression')
ensemble_client = Modelling.ModelClient.create_ensemble_from_models(
    meta_model=meta_client,
    bases=bases
)

In [91]:
ensemble_client.fit(X_train, y_train)

In [92]:
y_pred = ensemble_client.predict(X_valid)

In [93]:
y_pred

array([151790.09822852, 143217.45813573, 148113.82268225, 147049.21197371,
       145088.57974744, 138651.63556579, 131860.59685292, 126417.80984188,
       133045.44948108, 137372.47841164, 137192.7914511 , 148848.3641298 ])

In [94]:
len(y_pred)

12

In [61]:
def mape(y_true, y_pred):
    """
    Calcula el Mean Absolute Percentage Error (MAPE).

    Parámetros
    ----------
    y_true : array-like de shape (n_samples,)
        Valores reales.
    y_pred : array-like de shape (n_samples,)
        Valores predichos por el modelo.

    Devuelve
    -------
    float
        MAPE en porcentaje.
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Para evitar division por cero, añadimos un pequeño valor epsilon
    epsilon = np.finfo(np.float64).eps
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), epsilon))) * 100

In [95]:
mape(y_valid,y_pred)

np.float64(1.2715196923625853)

In [100]:
import plotly.express as px

In [101]:
df_pred = pd.DataFrame({
    "Fecha": df.loc[X_valid.index, "fecha"],
    "Real": y_valid.values,
    "Predicho": y_pred
})

# Gráfica interactiva
fig = px.line(df_pred, x="Fecha", y=["Real", "Predicho"],
              title="Predicción de corporativa_mn vs valores reales",
              labels={"value": "corporativa_mn", "Fecha": "Fecha", "variable": "Serie"})

fig.show()

In [102]:
base_preds = [model.predict(X_valid[features]) for model, features in bases]

In [103]:
base_mapes = [mape(y_valid, pred) for pred in base_preds]

In [104]:
base_mapes

[np.float64(16.1400232419508), np.float64(0.7040018011426735)]

In [105]:
y_pred_test = ensemble_client.predict(X_test)

In [106]:
results = X_test[['fecha']]
results['corporativa_mn'] = y_pred_test
results

/var/folders/97/kt6v_bd9753cp__4qp00b1h00000gr/T/ipykernel_39951/3246128320.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,fecha,corporativa_mn
184,2022-05-01,141718.468874
185,2022-06-01,140635.869770
186,2022-07-01,147484.422360
187,2022-08-01,95484.536286
188,2022-09-01,94793.993762
189,2022-10-01,92266.569239
190,2022-11-01,86263.656296
191,2022-12-01,85581.772848
192,2023-01-01,83995.389495
193,2023-02-01,88686.131769


In [107]:
results.to_csv('/Users/MI31539/Desktop/data_olympics/csv/predicciones_back_to_the_forecast.csv', index=False)